In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier, VotingClassifier
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

le = preprocessing.LabelEncoder()
imputer = Imputer()



train = pd.DataFrame(data=pd.read_csv('../input/train.csv'))
test = pd.DataFrame(data=pd.read_csv('../input/test.csv'))
print(train.columns)

labels = train.loc[:, 'Survived':'Survived'].values
train = train.loc[:, 'Pclass'::]

test.shape

In [ ]:
#print(train.isnull().sum())

train.Age = train.Age.fillna(train.Age.median())
train.Cabin = train.Cabin.fillna('unknown')
train.Embarked = train.Embarked.fillna(train.Embarked.mode().iloc[0])

#print(train.isnull().sum())
print('done')

In [ ]:
#print(test.isnull().sum())

test.Age = test.Age.fillna(train.Age.median())
test.Fare = test.Fare.fillna(test.Fare.median())
test.Cabin = test.Cabin.fillna('unknown')

#print(test.isnull().sum())
print('done')

In [ ]:

new_test = test.drop(columns=['PassengerId','Name','Ticket','Cabin'])
new_train = train.drop(columns=['Name','Ticket','Cabin'])
work = pd.concat([new_train,new_test])
work['Sex'] = work['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

work.loc[ work['Age'] <= 16, 'Age'] = 0
work.loc[(work['Age'] > 16) & (work['Age'] <= 32), 'Age'] = 1
work.loc[(work['Age'] > 32) & (work['Age'] <= 48), 'Age'] = 2
work.loc[(work['Age'] > 48) & (work['Age'] <= 64), 'Age'] = 3
work.loc[ work['Age'] > 64, 'Age'] = 4

#work['Embarked'] = work['Embarked'].map( {'S': 0, 'C': 2, 'Q':3} ).astype(int)
work['Embarked'] = pd.get_dummies(work['Embarked'])
#oh_work = pd.get_dummies(work)

#drop_work = oh_work.drop(columns=['PassengerId'])

print(work[work.Pclass == 1].sum())
print(work[work.Pclass == 2].sum())
print(work[work.Pclass == 3].sum())

work = work.drop(columns=['Fare','Embarked','SibSp'])


In [ ]:
#sel = VarianceThreshold(0.02)
#sel_work = pd.DataFrame(data=sel.fit_transform(oh_work))

#print(sel_work.shape)

new_train = work.iloc[:891] # get lengths and dont use these constants
new_test = work.iloc[891:] # get lengths and dont use these constants

print(new_train.shape)
print(labels.shape)
print(new_test.shape)

In [ ]:
#use k fold

#estimators = []
#for x in range(2, 100, 2):
 #   rfClf = RandomForestClassifier(n_estimators=100, max_depth=x,random_state=0)
 #   bag = BaggingClassifier(rfClf,max_samples=0.5, max_features=0.5)
 #   estimators.append(('x'+str(x),bag))

mlpClf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=100, alpha=1e-4, solver='sgd', verbose=10, tol=1e-4, random_state=1, learning_rate_init=.1)
svcClf = SVC(gamma='auto',probability=True)
rfClf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
gbClf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=10, random_state=0)

estimators = [('svc',svcClf),('rf',rfClf),('gb',gbClf)]

X_train, X_test, y_train, y_test = train_test_split(new_train, labels.ravel(), test_size=0.33, random_state=42)

eclf1 = VotingClassifier(estimators=estimators, voting='hard') 
eclf1.fit(new_train,labels.ravel())

#print(eclf1.score(X_train,y_train))
#print(eclf1.score(X_test,y_test))
print(eclf1.score(new_train,labels.ravel()))

results = eclf1.predict(new_test)

my_submission = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': results})
my_submission.to_csv('submission.csv', index=False)

print('complete')